In [1]:
import numpy as np

In [14]:
a = np.zeros((12,12), dtype=int)

In [19]:
xw1, yw1, zw1, x1, y1 = 6, 0, 2, 793, 884
xw2, yw2, zw2, x2, y2 = 0, 6, 2, 1080, 880
xw3, yw3, zw3, x3, y3 = 4, 0, 2, 845, 884
xw4, yw4, zw4, x4, y4 = 0, 4, 2, 1033, 880
xw5, yw5, zw5, x5, y5 = 6, 0, 0, 793, 950
xw6, yw6, zw6, x6, y6 = 0, 6, 0, 1080, 948

In [20]:
a[0] = [xw1, yw1, zw1, 1, 0, 0, 0, 0, -x1*xw1, -x1*yw1, -x1*zw1, -x1]
a[1] = [0, 0, 0, 0, xw1, yw1, zw1, 1, -y1*xw1, -y1*yw1, -y1*zw1, -y1]
a[2] = [xw2, yw2, zw2, 1, 0, 0, 0, 0, -x2*xw2, -x2*yw2, -x2*zw2, -x2]
a[3] = [0, 0, 0, 0, xw2, yw2, zw2, 1, -y2*xw2, -y2*yw2, -y1*zw2, -y2]
a[4] = [xw3, yw3, zw3, 1, 0, 0, 0, 0, -x3*xw3, -x3*yw3, -x3*zw3, -x3]
a[5] = [0, 0, 0, 0, xw3, yw3, zw3, 1, -y3*xw3, -y3*yw3, -y3*zw3, -y3]
a[6] = [xw4, yw4, zw4, 1, 0, 0, 0, 0, -x4*xw4, -x4*yw4, -x4*zw4, -x4]
a[7] = [0, 0, 0, 0, xw4, yw4, zw4, 1, -y4*xw4, -y4*yw4, -y4*zw4, -y4]
a[8] = [xw5, yw5, zw5, 1, 0, 0, 0, 0, -x5*xw5, -x5*yw5, -x5*zw5, -x5]
a[9] = [0, 0, 0, 0, xw5, yw5, zw5, 1, -y5*xw5, -y5*yw5, -y5*zw5, -y5]
a[10] = [xw6, yw6, zw6, 1, 0, 0, 0, 0, -x6*xw6, -x6*yw6, -x6*zw6, -x6]
a[11] = [0, 0, 0, 0, xw6, yw6, zw6, 1, -y6*xw6, -y6*yw6, -y6*zw6, -y6]

In [22]:
a

array([[    6,     0,     2,     1,     0,     0,     0,     0, -4758,
            0, -1586,  -793],
       [    0,     0,     0,     0,     6,     0,     2,     1, -5304,
            0, -1768,  -884],
       [    0,     6,     2,     1,     0,     0,     0,     0,     0,
        -6480, -2160, -1080],
       [    0,     0,     0,     0,     0,     6,     2,     1,     0,
        -5280, -1768,  -880],
       [    4,     0,     2,     1,     0,     0,     0,     0, -3380,
            0, -1690,  -845],
       [    0,     0,     0,     0,     4,     0,     2,     1, -3536,
            0, -1768,  -884],
       [    0,     4,     2,     1,     0,     0,     0,     0,     0,
        -4132, -2066, -1033],
       [    0,     0,     0,     0,     0,     4,     2,     1,     0,
        -3520, -1760,  -880],
       [    6,     0,     0,     1,     0,     0,     0,     0, -4758,
            0,     0,  -793],
       [    0,     0,     0,     0,     6,     0,     0,     1, -5700,
            0,     0

In [25]:
u, s, vh = np.linalg.svd(a, full_matrices=True)
u.shape, s.shape, vh.shape

((12, 12), (12,), (12, 12))

In [26]:
vh

array([[ -1.10794458e-04,  -5.00454917e-04,  -1.47859775e-04,
         -1.12077944e-04,  -1.24572353e-04,  -4.23568432e-04,
         -1.30248676e-04,  -1.00489305e-04,   2.02181908e-01,
          9.20316356e-01,   2.64604042e-01,   2.05242448e-01],
       [  5.11021336e-04,  -1.51343656e-04,   9.19559809e-05,
          6.68441935e-05,   5.79116126e-04,  -1.28618915e-04,
          1.07440272e-04,   8.26256256e-05,  -9.38220634e-01,
          2.80059028e-01,  -1.62645632e-01,  -1.21879915e-01],
       [ -7.85007509e-05,  -7.38857974e-05,   5.29682521e-04,
          3.19708097e-05,  -1.58681211e-04,  -9.16704412e-05,
          4.97491456e-04,   1.20645841e-05,   2.34540937e-01,
          2.25846911e-01,  -9.45170626e-01,  -2.52117872e-02],
       [ -1.79913254e-04,  -1.63779540e-04,  -2.47682525e-05,
          5.17915680e-04,  -3.06195725e-04,   1.00963113e-04,
         -5.44323754e-05,   5.05427499e-04,   1.54448496e-01,
          1.53549451e-01,   1.00910807e-01,  -9.70764929e-01],
    

In [33]:
r = vh[:, 11]
r

array([  2.05242448e-01,  -1.21879915e-01,  -2.52117872e-02,
        -9.70764929e-01,   6.46144836e-05,  -1.86273606e-04,
         4.59834716e-05,  -7.83985181e-05,   1.16421188e-04,
         1.37568662e-04,  -7.09415188e-05,   7.47369018e-04])

In [47]:
result = r.reshape(3, 4)
result

array([[  2.05242448e-01,  -1.21879915e-01,  -2.52117872e-02,
         -9.70764929e-01],
       [  6.46144836e-05,  -1.86273606e-04,   4.59834716e-05,
         -7.83985181e-05],
       [  1.16421188e-04,   1.37568662e-04,  -7.09415188e-05,
          7.47369018e-04]])

In [48]:
b = result[:, 3]
B = result[:, [0,1,2]]
b


array([ -9.70764929e-01,  -7.83985181e-05,   7.47369018e-04])

In [49]:
B

array([[  2.05242448e-01,  -1.21879915e-01,  -2.52117872e-02],
       [  6.46144836e-05,  -1.86273606e-04,   4.59834716e-05],
       [  1.16421188e-04,   1.37568662e-04,  -7.09415188e-05]])

In [51]:
A = B*B.transpose()
A

array([[  4.21244623e-02,  -7.87520777e-06,  -2.93518620e-06],
       [ -7.87520777e-06,   3.46978563e-08,   6.32588465e-09],
       [ -2.93518620e-06,   6.32588465e-09,   5.03269909e-09]])

In [58]:
mu0 = A[0,2]
v0 = A[1,2]
beta = np.sqrt(A[1,1]-np.square(v0))
alfa = np.sqrt(A[0,0] - np.square(mu0))

In [78]:
K = [[alfa, 0, mu0],[0, beta, v0],[0, 0, 1]]
K = np.matrix(K)
Km = K.I
Km1 = np.squeeze(np.asarray(Km))

In [79]:
rotation = Km1*B
t = Km1*b